In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Import library
import pandas as pd
import numpy as np
import random as rd
import math

In [ ]:
df = pd.read_excel("/content/drive/MyDrive/Đàn kiến không time out/Đàn kiến không time out/data.xlsx")
df_distance = pd.read_excel('/content/drive/MyDrive/Đàn kiến không time out/Đàn kiến không time out/distance-matrix.xlsx')
df_distance


# Distance matrix

In [ ]:
# Get n ant is quantity customer
n = df['Open'].index[-1]
n = n + 1
m = n
# Put coordinates on gg maps to get distance
dfAddr = df['Coordinates Valid'] # dataframe

# Function

In [ ]:
# Hàm tính toán khoảng cách
def calcDistAuto(coord1, coord2):
    """
    Tính toán khoảng cách giữa hai tọa độ.
    """
    return np.linalg.norm(np.array(coord1) - np.array(coord2))

In [ ]:

# Convert dataframe customer to array one dimensional
listCustomerAddress = df['Coordinates Valid'][0:].to_list()

# Distance from random place to other place

listDistance = []
# placeI ="Mỹ Yên, Bến Lức, Long An"
# placeI = "Phường 2, Tân An, Long An"
def getListDistance(placeI, listCustomerAddress):
    for cusAddr in listCustomerAddress:
        # print(cusAddr)
        dist = calcDistAuto(placeI, cusAddr)
        listDistance.append(dist)
    return listDistance

# Function get total distance for node i
def getTotalDistanceForOneNode(listDistance):
    return round(sum(listDistance), 2)

speed = 50 # (km/h)
# Example
# place1 = "Phường 2, Tân An, Long An"
# place2 = "Mỹ Yên, Bến Lức, Long An"

# distance = calcDistAuto(place1, place2) # (km)

def calculateDuration(speed, distance, route):

    waitingtime = len(route)*0.5
    totalhour = distance/speed + waitingtime
    return totalhour # (hour)

def printDuration(total_hours):
      # Tách phần nguyên và phần thập phân của giờ
    hours = int(total_hours)
    minutes = (total_hours - hours) * 60

  # Làm tròn số phút
    minutes = round(minutes)

    print(str(hours) + "h " + str(round(minutes)) + "m")

listDuration = []
def getListDuration(placeI, listCustomerAddress,speed):
    for cusAddr in listCustomerAddress:
        # print(cusAddr)
        dist = calcDistAuto(placeI, cusAddr)
        listDuration.append(dist/speed + 0.5)
    return listDuration


# Greed


In [ ]:
def findJForI_greed(listDistance, index, listplace):
    # have: listDistance, listH2, alpha, beta
    valueRandom = df.iloc[index, df.columns.get_loc('Coordinates Valid')]
    result = 0 # it is index of j
    # done
    # Get maximum odor trace concentration
    maxOdor = float('inf')
    for i in range(len(listDistance)):
        if (listDistance[i] != 0) & (listDistance[i] < maxOdor):
            maxOdor = listDistance[i]
            result = listplace[i]

    return result

listCustomerAddress = df["Coordinates Valid"][0:].to_list()
placeI = df["Coordinates Valid"][0]

ori_road = listCustomerAddress
roadSavedNodes_greed = []

listDistance=[]

while ori_road != []:

  listDistance = getListDistance(placeI, ori_road)
  # print(listDistance)
  # roadSavedNodes.clear()
  # roadSavedNodes.append(currentIndex)

  current_index = df[df["Coordinates Valid"] == placeI].index[0]

  next_index_place = findJForI_greed(listDistance, current_index, ori_road)

  # next_index = df[df["Coordinates Valid"] == next_index_place].index[0]
  # print(next_index)

  roadSavedNodes_greed.append(current_index)
  ori_road.remove(placeI)

  placeI = next_index_place

  listDistance.clear()

roadSavedNodes_greed

capacitylist = df["Capacity"].to_list()

In [ ]:
currentcapacity = 50
capacitymax = 2000
durationmax = 48
currentduration = 12
listCustomerAddress = df["Coordinates Valid"][0:].to_list()
def findJForI(listDistance, index,
              currentcapacity, capacitylist, capacitymax,
              currentduration, listDuration, durationmax, odor_matrix, alpha, beta):
    # have: listDistance, listH2, alpha, beta
    # alpha = 1
    # beta = 2
    valueRandom = df.iloc[index, df.columns.get_loc('Coordinates Valid')]
    arrOdor = []
    result = 0 # it is index of j
    i=0
    odor_list = []
    # odor = 2
    # Creat array for odor
    for i in range(len(listDistance)):
        odorvalue = odor_matrix[index][i] # distance
        odor_list.append(odorvalue)

    odor_arr = pd.array(odor_list)
    # print(odor_arr)
    # Calculating array storage heuristic value
    for i in range(len(listDistance)):
        h1 = listDistance[i] # distance

        if (h1) == 0: # because it is start node
            arrOdor.append(0)
            continue
        o = 50000/(h1)
        arrOdor.append(o)

    heuristic_mattrix = pd.array(arrOdor)
    # print(arrOdor)

    denominator = sum(pow(odor_arr, alpha) * pow(heuristic_mattrix, beta)) # const # done
    # print(denominator)
    numerator = 0

    # Calculate odor
    arrOdorChooseJ = []
    for i in range(len(listDistance)): # correct is n, n = listDistance = listH2
        numerator = pow(odor_arr[i], alpha) * pow(heuristic_mattrix[i], beta) # variable base
        if denominator == 0:
            denominator = 0.00000000001
        temp = numerator / denominator
        arrOdorChooseJ.append(temp)

    # print(arrOdorChooseJ)
    # Choose wh meet capacity
    listwh = []
    for i in range(len(listDistance)):
        if (currentcapacity + capacitylist[i] <= capacitymax) & (listDistance[i] != 0) & (currentduration + listDuration[i] <= durationmax):
            listwh.append(i)

    # print(listwh)
    # Get maximum odor trace concentration
    maxOdor = 0
    if listwh != []:
        for i in listwh:
          if (arrOdorChooseJ[i] > maxOdor):
                maxOdor = arrOdorChooseJ[i]
                result = i

    else:
        for i in range(len(listDistance)):
          if (arrOdorChooseJ[i] > maxOdor):
            maxOdor = arrOdorChooseJ[i]
            result = i

    # print(arrOdorChooseJ)
    # print(listwh)
    return result



In [ ]:
def notTravaled(odor_matrix, evap_coef=0.1):
	return odor_matrix * (1 - evap_coef)

In [ ]:
def defaultOdorMatrix(x, n):
    """
    Khởi tạo ma trận mùi với NumPy.
    """
    # Tạo một ma trận nxn toàn số `x`
    odorMatrix = np.full((n, n), x)

    # Đặt đường chéo chính thành 0
    np.fill_diagonal(odorMatrix, 0)

    # Chuyển đổi sang DataFrame (nếu cần)
    return pd.DataFrame(odorMatrix)

In [ ]:
def totalDistanceVisited(road, distance_matrix):
    """
    Tính tổng quãng đường đã đi qua sử dụng ma trận khoảng cách.
    road: danh sách các điểm đi qua.
    distance_matrix: ma trận khoảng cách giữa các điểm.
    """
    # Lấy chỉ số các cặp (road[i], road[i+1])
    indices = zip(road[:-1], road[1:])

    # Sử dụng NumPy để tính tổng khoảng cách
    total_distance = sum(distance_matrix[i, j] for i, j in indices)

    # Thêm quãng đường quay lại điểm bắt đầu
    total_distance += distance_matrix[road[-1], road[0]]

    return round(total_distance, 2)

# Total distance greed

In [ ]:
listCustomerAddress = df["Coordinates Valid"][0:].to_list()
roadSavedNodes_greed
finalTotal = 0
total_distance_greed = 0
centerPlace = df.loc[0, 'Coordinates Valid']
total_distance_greed = totalDistanceVisited(roadSavedNodes_greed)
val = roadSavedNodes_greed[-1]
total_distance_greed += calcDistAuto(centerPlace, listCustomerAddress[val])

total_distance_greed

In [ ]:
num_cus = len(listCustomerAddress) -1

In [ ]:
def determineAddEdges(roadSavedNodes):
    """
    Tạo danh sách các cạnh cần cập nhật mùi bằng NumPy.
    """
    # Chuyển danh sách thành mảng NumPy
    road_array = np.array(roadSavedNodes)

    # Lấy các cặp cạnh
    edges = np.column_stack((road_array[:-1], road_array[1:]))

    # Thêm cạnh quay lại điểm đầu
    reverse_edges = edges[:, ::-1]

    return np.vstack((edges, reverse_edges))

In [ ]:
def randomNumber(n):
    list_ran = list(range(0,n))
    rd.shuffle(list_ran)
    value = list_ran[0]
    return value
